In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


# Setup stuff: Connectivity

In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Actual analytics ..

In [ ]:

#  View the graph
#

l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()


In [ ]:

from katana.remote import analytics

print("--")


In [ ]:

#  Getting the airport node id
#

def f_get_airport(i_arg1):
    
   l_query  = """
      MATCH (n: Airport) 
      WHERE n.airport_code = '{0}'
      RETURN ID(n) AS my_id
      """.format(i_arg1)
   
   l_id = my_graph.query(l_query)["my_id"][0]

   print("Airport id:  %s   %d" % (i_arg1, l_id))
  
   return (str(l_id))


print("--")


In [ ]:

#  Running analytics.ksssp() ..



l_airport     = "MKE"
#  l_airport     = "ORD"
#  l_airport     = "DEN"
#  l_airport     = "SJC"
   #
l_airport_id  = f_get_airport(l_airport)


l_walk_length         = 10
l_number_of_walks     = 10


l_result = analytics.random_walks(
   my_graph,
      #
   walk_length       = l_walk_length,
   number_of_walks   = l_number_of_walks,
      #
   root_nodes        = l_airport_id 
   )

print("--")

#  Error received,
#
#     ---------------------------------------------------------------------------
#     InvalidRequestError                       Traceback (most recent call last)
#     Cell In [9], line 17
#          13 l_walk_length         = 10
#          14 l_number_of_walks     = 10
#     ---> 17 l_result = analytics.random_walks(
#          18    my_graph,
#          19       #
#          20    walk_length       = l_walk_length,
#          21    number_of_walks   = l_number_of_walks,
#          22       #
#          23    root_nodes        = l_airport_id 
#          24    )
#          26 print("--")
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:168, in async_to_sync.<locals>.wrapper(timeout, *args, **kwargs)
#         165 if convert_objects:
#         166     registry = AsyncToSyncClassRegistry.get()
#         167     return registry.async_to_sync(
#     --> 168         wait_for(
#         169             async_func(
#         170                 *(registry.sync_to_async(a) for a in args),
#         171                 **{k: registry.sync_to_async(v) for k, v in kwargs.items()},
#         172             ),
#         173             timeout=timeout,
#         174         )
#         175     )
#         176 return wait_for(async_func(*args, **kwargs), timeout=timeout)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:147, in wait_for(coro, timeout)
#         145 try:
#         146     future = asyncio.run_coroutine_threadsafe(timeout_coro, loop=AsyncRunnerThread.get().loop)
#     --> 147     return future.result()
#         148 except KeyboardInterrupt:
#         149     inner_future.cancel()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:444, in Future.result(self, timeout)
#         442     raise CancelledError()
#         443 elif self._state == FINISHED:
#     --> 444     return self.__get_result()
#         445 else:
#         446     raise TimeoutError()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:389, in Future.__get_result(self)
#         387 if self._exception:
#         388     try:
#     --> 389         raise self._exception
#         390     finally:
#         391         # Break a reference cycle with the exception in self._exception
#         392         self = None
#     
#     File /opt/conda/lib/python3.8/asyncio/tasks.py:455, in wait_for(fut, timeout, loop)
#         450     warnings.warn("The loop argument is deprecated since Python 3.8, "
#         451                   "and scheduled for removal in Python 3.10.",
#         452                   DeprecationWarning, stacklevel=2)
#         454 if timeout is None:
#     --> 455     return await fut
#         457 if timeout <= 0:
#         458     fut = ensure_future(fut, loop=loop)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/analytics/__init__.py:409, in random_walks(graph, walk_length, number_of_walks, root_nodes, backward_probability, forward_probability, is_uniform, is_node2vec, use_gpu)
#         406 if no_random_walks:
#         407     return DataFrame()
#     --> 409 return await graph._run_asynchronously_and_read_from_parquet(data)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/graph.py:235, in Graph._run_asynchronously_and_read_from_parquet(self, data)
#         232 async def _run_asynchronously_and_read_from_parquet(self, data) -> DataFrame:
#         233     data = self._add_operation_metadata(data)
#     --> 235     result = await self._client._run_on_graph_and_wait(self, data)
#         237     files = result["result_files"]
#         238     operation_id = result["operation_id"]
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:323, in Database._run_on_graph_and_wait(self, graph, data)
#         321 max_attempts = 60
#         322 attempt = 0
#     --> 323 op = await self._run_on_graph(graph, data)
#         324 while attempt < max_attempts:
#         325     try:
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:317, in Database._run_on_graph(self, graph, data)
#         315 if self._txn:
#         316     return await self._txn._run_on_graph(graph, data)
#     --> 317 return await self._client.run_on_graph(graph.graph_id, data)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:108, in _GraphClient.run_on_graph(self, graph_id, data)
#         107 async def run_on_graph(self, graph_id: str, data):
#     --> 108     return await self._json_request(f"graph/{graph_id}/run", "POST", json=data)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:386, in AsyncClient._json_request(self, endpoint, method, json, params, timeout)
#         384 async def _json_request(self, endpoint: str, method: str, *, json=None, params=None, timeout=None) -> dict:
#         385     """Makes a request and decodes its response into a JSON object."""
#     --> 386     async with self._stream(endpoint, method, json=json, params=params, timeout=timeout) as resp:
#         387         await resp.aread()
#         388         return resp.json()
#     
#     File /opt/conda/lib/python3.8/contextlib.py:171, in _AsyncGeneratorContextManager.__aenter__(self)
#         169 async def __aenter__(self):
#         170     try:
#     --> 171         return await self.gen.__anext__()
#         172     except StopAsyncIteration:
#         173         raise RuntimeError("generator didn't yield") from None
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:438, in AsyncClient._stream(self, endpoint, method, json, params, timeout)
#         436         else:
#         437             await resp.aread()
#     --> 438             self._raise_error(resp)
#         439 except (httpx.RequestError, httpx.RemoteProtocolError, httpx.ConnectError) as exc:
#         440     if not isinstance(exc, httpx.RequestError):
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:410, in AsyncClient._raise_error(cls, resp)
#         407     message = httpx.codes.get_reason_phrase(code)
#         409 if code == httpx.codes.BAD_REQUEST:
#     --> 410     raise errors.InvalidRequestError(url, code, message, headers)
#         411 # codes.UNAUTHORIZED (401) for invalid/missing credentials (i.e. authentication error).
#         412 if code == httpx.codes.UNAUTHORIZED:
#     
#     InvalidRequestError: b'{"code":400,"message":"json: cannot unmarshal string into Go struct field RandomWalksParameters.analytics.random_walks.root_nodes of type []uint32"}'



In [ ]:

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  l_result = my_graph.query(
#     """
#     MATCH (n) - [r] -> (m)
#     RETURN r
#     """ )
#  print(tabulate(l_result, headers='keys', tablefmt='psql'))
